# **Proyecto Final Análisis Aplicado**
## Otoño 2020

*   José Luis Cordero Rodríguez
*   Tonantzin Real Rojas
*   Francisco Velasco Medina



In [ ]:
import numpy as np
import math as ma
import pandas as pd
import random as rd
from numpy import linalg as la
import cmath
import warnings

### Clase, algoritmos y métodos

En esta sección presentamos la clase **MetodosPF** que contiene los siguientes métodos y algoritmos:

Métodos utilizados:
1.   **grad:** método para calcular el gradiente de una función
2.   **hess:** método para calcular el hessiana de una función
3.   **norma:** método para calcular la norma 2 de un vector
4.   **cholesky:** método para volver suficientemente positiva a la hessiana y poder aplicarle la descomposición Cholesky

Algoritmos requeridos en el proyecto:
1.   **backS:** algoritmo que corresponde a la búsqueda lineal base
2.   **newton:** algoritmo que corresponde al algoritmo de Newton
3.   **newtonMod:** algoritmo que corresponde al algoritmo de Newton modificado
4.   **bfgs:** algoritmo que corresponde al algoritmo BFGS

In [ ]:
class MetodosPF:
    def __init__(self,f, xk, pk, eps = 0.00001):
        try:
            self.f = f
            self.xk = xk
            self.pk = pk
            self.eps = eps
        except:
            return "Debe ser función multivariada"
    
    # POO
    def setXk(self, xk):
        self.xk = xk
    
    def setF(self, f):
        self.f = f
        
    ## Métodos ## 
    # Método para calcular el gradiente
    def grad(self,x=None):
        if x is None:
          x = self.xk  
        n = len(x)
        res = np.zeros(n)
        for i in range(n):
            zer = np.zeros(n)
            zer[i] += self.eps
            x1 = x + zer
            res[i] = (self.f(x1) - self.f(x)) / self.eps
        return res

    # Método para calcular la hessiana
    def hess(self,xk):
      n = len(xk)
      res = np.zeros((n,n))
      for i in range(n):
          for j in range(n):
              zer = np.zeros(n)
              zer2 = np.zeros(n)
              zer[i] += self.eps
              zer2[j] += self.eps
              x_e = xk + zer + zer2
              x_ei = xk + zer
              x_ej = xk + zer2
              res[i][j] = (self.f(x_e) - self.f(x_ei) - self.f(x_ej) + self.f(xk)) / (self.eps**2)
      return res

    # Método para sacar la norma
    def norma(self,vector):
      suma = 0
      for i in range(len(vector)):
        suma = suma+ma.pow(vector[i],2)
      return ma.sqrt(suma)

    # Algoritmo de Cholesky
    def cholesky(self, A, b, k):
      t = 0
      if min(np.diag(A)) > 0:
          t = 0
      else: 
          t = -min(np.diag(A)) + b 
      for j in range(k):
          try: 
              L = la.cholesky(A + t*np.identity(len(A)))
          except:
              t = max(2*t,b)
          else:
              break
      return np.dot(L,L)
    
    ## Algoritmo ##
    # Algoritmo de Búsqueda Lineal Base (Algoritmo de búsqueda de paso)
    def backS(self,a, xk ,pk, seed = 123):
        np.random.seed(seed)
        c = np.random.rand()   
        while self.f(xk + a * pk) > self.f(xk) + c * a * np.dot(self.grad(xk), pk):
            rho = np.random.rand()
            a = rho * a
        return a

    # Algoritmo de Newton sin modificación a la hessiana y con tamaño de paso fijo
    def newton(self,alpha,n=100):
        xk = self.xk
        for i in range(n):
            Bk = self.hess(xk)
            pk = np.dot(la.inv(Bk),-1 * self.grad(xk))
            xk = xk + alpha * pk
        return xk

    # Algoritmo de Newton con modificación a la hessiana
    def newtonMod(self, n=100, k=100, b=2):
      xk = self.xk
      for i in range(n):
          Bk = self.hess(xk)
          try:
              L=la.cholesky(Bk)
          except:
              Bk = self.cholesky(Bk, b, k)
          pk = np.dot(la.inv(Bk),-1 * self.grad(xk))
          xk = xk + self.backS(1,xk,pk) * pk
      return xk

    # Algoritmo BFGS
    def bfgs(self,umbral=3,e=0.1):
      xk = self.xk
      pk = self.pk
      k= 0
      sk = np.array(self.backS(umbral,xk,pk))*pk 
      yk = self.grad(xk+sk)-self.grad()
      H0 = (np.dot(yk,sk)/np.dot(yk,yk))*np.identity(len(xk))
      pk = -np.dot(H0,self.grad())
      xk_1 = xk

      while self.norma(self.grad(xk_1))> e:
        pk = -np.dot(la.inv(self.hess(xk_1)),self.grad(xk_1)) 
        xk_1 = xk_1 + self.backS(umbral,xk_1,pk)*pk
        sk = self.backS(umbral,xk_1,pk)*pk
        yk = self.grad(xk_1)-self.grad(xk_1-sk)
        pk = 1/(np.dot(yk,sk))
        Hk_1 = np.dot(np.dot((np.identity(len(xk_1))-pk*np.dot(sk,yk)),self.hess(xk_1)),np.identity(len(xk_1))-pk*np.dot(sk,yk)) + pk*np.dot(sk,sk)
        k = k+1 
      return xk_1 


### Función polinomial para corroborar que funciona

In [ ]:
def p2():
    return lambda x: x[0]**2 + 3*x[1]**3

In [ ]:
xk = np.array([2,0])
pk = np.array([0.01, 0.02])

metodos = MetodosPF(p2(),xk, pk)
metodos.grad([3,0])
# metodos.hess(xk)
# metodos.backS(2,xk,pk)
# metodos.newton(0.1)
# metodos.newtonMod()
# metodos.bfgs(1)

array([6.00001000e+00, 3.55271368e-10])

### Función Rosenbrock

In [56]:
def rosen(a=1, b=100):
    return lambda x: (a - x[0]) ** 2 + b * (x[1] - x[0] ** 2) ** 2

In [57]:
xk = np.array([0.8,1])
pk = np.array([2, 5])
metodos = MetodosPF(rosen(),xk, pk)
# metodos.grad()
metodos.hess(xk)
# metodos.backS(2,xk,pk)
# metodos.newton(0.1)
# metodos.newtonMod()
# metodos.bfgs()

array([[ 370.01916198, -320.00199823],
       [-320.00199823,  199.99998102]])

### Crímenes y cámaras

In [58]:
def crim(x):
  crimenes = pd.read_csv('crime_data.csv')
  crimenes = crimenes[0:x]

  maxLat = round(crimenes['lat'].max(),2)
  maxLong = round(crimenes['long'].max(),2)
  minLat = round(crimenes['lat'].min(),2)
  minLong = round(crimenes['long'].min(),2)
  
  crimenes['nLat'] = (crimenes['lat']-minLat)/(maxLat - minLat)
  crimenes['nLong'] = (crimenes['long']-minLong)/(maxLong - minLong)

  coord = crimenes[['nLong','nLat']].to_numpy()
  return lambda camaras: costo2(camaras, coord)


def costo2(camaras, coord):  
  c = 0
  
  for i in range(int(len(camaras)/2)):
  	for j in range(len(coord)):
  		c += (camaras[i] - coord[j][0])**2 + (camaras[i+int(len(camaras)/2)] - coord[j][1])**2
			
  for i in range(int(len(camaras)/2)):
  	for j in range(int(len(camaras)/2)):
  		if i != j:
  			 c += 1 / ((camaras[i] - camaras[j])**2 + (camaras[i+int(len(camaras)/2)] - camaras[j + int(len(camaras)/2)])**2)

  return round(c,2)

Lectura de datos

In [59]:
crimenes = pd.read_csv('crime_data.csv')
crimenes.head()
crim240=crimenes.head(240)
crim240.head()
print(len(crim240))

240


Aplicamos los algoritmos programados

In [60]:
camara = [0.5,0.4,0.2,0.7]
h=crim(24000)
h(camara)

7088.02

In [ ]:
pk = [0.1,0.1]
metodos = MetodosPF(h,camara, pk)
aNewton = metodos.newtonMod()
aNewton

In [ ]:
h(aNewton)

2844.84

In [ ]:
np.random.seed(123)
v = np.random.rand(16)
print(v)
g=crim(24)
print(g(v))

[0.69646919 0.28613933 0.22685145 0.55131477 0.71946897 0.42310646
 0.9807642  0.68482974 0.4809319  0.39211752 0.34317802 0.72904971
 0.43857224 0.0596779  0.39804426 0.73799541]
1657.46


In [ ]:
pk = 0.05*np.ones(16)
metodos = MetodosPF(crim(24),v, pk)
aNewton = metodos.newtonMod()
aNewton

array([ 0.6949824 ,  0.28733173,  0.22567458,  0.55121543,  0.72111498,
        0.42307695,  0.98089648,  0.68482633,  0.49433477,  0.40517972,
        0.34196976, -0.4437965 ,  0.43492962,  0.05875521,  0.39718501,
        0.73728065])

In [ ]:
print(g(aNewton))

1043.5
